In [1]:
import sys
import os
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
# import preprocessing.preprocess
!pwd
# from legendary_game_recs.preprocessing import preprocess
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
from preprocessing.preprocess import *

/Users/antonis/code/Ant-mel/legendary_game_recs/notebooks


In [2]:
raw_data = pd.read_csv('../raw_data/all_data_batch1_30to60k')
raw_2 = pd.read_csv('../raw_data/all_data_batch1_0to4k')
raw_3 = pd.read_csv('../raw_data/all_data_batch2_4kto8k')
raw_4 = pd.read_csv('../raw_data/all_data_batch3_8kto12k')

In [3]:
data = pd.concat((raw_2, raw_3, raw_4, raw_data))

In [58]:
data['developers'].apply(make_stringlist_list)

2010      [Frontier Works,  Abracadabra]
2011                                  []
2012                             [Darso]
2013                    [IO Interactive]
2014                                  []
                      ...               
3005               [Grinding Gear Games]
3006    [Petite Games,  Ratalaika Games]
3007                                  []
3008          [Male Doll,  EastAsiaSoft]
3009                                  []
Name: developers, Length: 1000, dtype: object

In [66]:
list_of_str = ['Ltd.', 'salmon', 'chicken']


if 'Ltd.' in list_of_str:
    print('YES')
if 'Inc.' in list_of_str:
    print('YES')
if 'LLC' in list_of_str:
    print('YES')

YES


In [ ]:
def make_stringlist_list(string):
    '''This removes square brackets, and splits the string by comma to the create a list'''
    list_of_strings = string[2:-2].replace("'", '').split(',')
    return list_of_strings

def remove_whitespace(list_):
    '''This removes whitesapces from items within lists'''
    empty = []

    for i in range(len(list_)):
        item = list_[i].strip()
        empty.append(item)
    return empty

def clean_stringlists(df):
    '''For lists that were imported as strings, this removes square brackers and cleans up trailing whitespaces'''
    string_to_list_df = df.apply(make_stringlist_list)
    remove_whitespace_df = string_to_list_df.apply(remove_whitespace)

    return remove_whitespace_df

# This is the final one mentioned above
def make_list_columns_to_lists(df, columns):
    '''This returns a dataframe of columns where lists where imported as strings, and returns them to their list state'''
    cleaned_df = pd.DataFrame()

    for col in columns:
        cleaned = clean_stringlists(df[col])
        cleaned_df[col] = cleaned
    return cleaned_df

In [44]:
categories_as_lists = make_list_columns_to_lists(data, ['developers', 'platforms', 'genres'])

In [45]:
categories_as_lists

,developers,platforms,genres
0,[PilotXross],[SteamVR],"[Indie, Racing, Sport]"
1,[],[Windows PC],"[Adventure, Indie]"
2,[Pathea Games],[PlayStation 5],[RPG]
3,"[8 Million Bit, Hanoi Studios]",[Windows PC],[]
4,[],[Windows PC],"[Adventure, Racing]"
...,...,...,...
29954,[Jesse Makkonen],[Windows PC],"[Adventure, Indie, Puzzle]"
29955,"[Peter Moorhead, Curve Digital]","[Windows PC, Mac, Linux]","[Adventure, Indie, Point-and-Click, Puzzle, St..."
29956,[],"[Android, iOS]","[Puzzle, RPG]"
29957,[Serious Games Interactive],"[Windows PC, Mac]","[Adventure, Indie]"


In [46]:
genres = categories_as_lists['genres']

In [47]:
encoded_genre, genre_encoder = categorical_encoder(genres)

In [48]:
encoded_genre.sum().sort_values(ascending=False)

Indie                  21313
Adventure              16347
RPG                     7405
Simulator               7315
Strategy                6472
Puzzle                  4677
Arcade                  3937
Shooter                 3819
Platform                3370
                        2696
Visual Novel            2372
Sport                   1995
Racing                  1500
Fighting                1069
Point-and-Click          932
Brawler                  889
Turn Based Strategy      862
Card & Board Game        700
Tactical                 669
Music                    634
Real Time Strategy       473
Quiz/Trivia              209
Pinball                  128
MOBA                      28
dtype: int64

In [49]:
dev = categories_as_lists['developers']

encoded_dev, dev_encoder = categorical_encoder(dev)

devs_by_power = pd.DataFrame(encoded_dev.sum().sort_values(ascending=False))

In [50]:
devs_by_power.head(50)

,0
,16092
Square Enix,258
Nintendo,252
Sega,248
Bandai Namco Entertainment,228
Ubisoft Entertainment,193
HAMSTER,185
Big Fish Games,177
Inc.,158
Electronic Arts,143


,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,image,url
1,Plague Hunter,2029-02-07,0,0,2,7,[],0.0,"['Adventure', 'Indie']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/plague-hunter/
7,Penny Blood,2025-12-31,1,0,24,117,[],0.0,['RPG'],"['Windows PC', 'PlayStation 5', 'Xbox Series']",...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/penny-blood/
8,Genimas: Life Reborn,2025-12-31,0,0,0,0,[],0.0,"['Adventure', 'Indie', 'RPG']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/genimas-life-r...
4,Riversiders,2025-12-31,1,0,0,0,[],0.0,"['Adventure', 'Racing']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/riversiders/
12,UpRooted,2025-10-01,0,0,2,2,[],0.0,"['Adventure', 'Indie']",['Windows PC'],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/uprooted/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29402,Rainbow Six Siege - Pro League Bandit Set,0001-01-01,0,0,0,0,[],0.0,[],[],...,0,0,0,0,0,0,0,0,NaN,https://www.backloggd.com/games/rainbow-six-si...
29386,Rainbow Six Siege - Pro League Bandit Set,0001-01-01,0,0,0,0,[],0.0,[],[],...,0,0,0,0,0,0,0,0,NaN,https://www.backloggd.com/games/rainbow-six-si...
15473,Sok,0001-01-01,0,0,0,0,[],0.0,[],[],...,0,0,0,0,0,0,0,0,https://images.igdb.com/igdb/image/upload/t_co...,https://www.backloggd.com/games/sok/
12251,Tom Clancy's Rainbow Six Siege: Operation Para...,0001-01-01,0,0,0,0,[],0.0,[],[],...,0,0,0,0,0,0,0,0,NaN,https://www.backloggd.com/games/tom-clancys-ra...


In [26]:
platforms = categories_as_lists['platforms']

encoded_plat, plat_encoder = categorical_encoder(platforms)

platforms_encoded = pd.DataFrame(encoded_plat.sum().sort_values(ascending=False))

In [27]:
platforms_encoded.head(20)

,0
Windows PC,22876
Mac,6536
PlayStation 4,4958
Nintendo Switch,4617
Linux,4277
Xbox One,4034
iOS,3355
Android,3094
Web browser,1014
PlayStation Vita,722


In [ ]:
def make_stringlist_list(string):
    '''This removes square brackets, and splits the string by comma to the create a list'''
    list_of_strings = string[2:-2].replace("'", '').split(',')
    return list_of_strings

def remove_whitespace(list_):
    '''This removes whitesapces from items within lists'''
    empty = []
    
    for i in range(len(list_)):
        item = list_[i].strip()
        empty.append(item)
    return empty

def clean_stringlists(df):
    '''For lists that were imported as strings, this removes square brackers and cleans up trailing whitespaces'''
    string_to_list_df = df.apply(make_stringlist_list)
    remove_whitespace_df = string_to_list_df.apply(remove_whitespace)
    
    return remove_whitespace_df

In [ ]:
def make_list_columns_to_lists(df, columns):
    '''This returns a dataframe of columns where lists where imported as strings, and returns them to their list state'''
    cleaned_df = pd.DataFrame()
    
    for col in columns:
        cleaned = clean_stringlists(df[col])
        cleaned_df[col] = cleaned
    return cleaned_df

In [ ]:
updated_df = make_list_columns_to_lists(data, ['genres', 'platforms', 'developers'])

In [ ]:
updated_df

In [ ]:
mlb_genre = MultiLabelBinarizer()
mlb_platform = MultiLabelBinarizer()
mlb_developer = MultiLabelBinarizer()


transfomed_genre = mlb_genre.fit_transform(updated_df['genres'])
transformed_platform = mlb_platform.fit_transform(updated_df['platform'])
transfomed_developer = mlb_developer.fit_transform(updated_df['developer'])

In [ ]:
genre_encoded = pd.DataFrame(transfomed, columns=mlb.classes_)

In [ ]:
def categorical_encoder(category):
    mlb = MultiLabelBinarizer()
    transformed = mlb.fit_transform(category)
    
    df = pd.DataFrame(transformed, columns=mlb.classes_)
    
    return df, mlb


In [ ]:
df, mlb_thing = categorical_encoder(updated_df['developers'])

In [ ]:
df.shape

In [ ]:
genre_encoded